In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

ERROR: datasets 2.21.0 has requirement requests>=2.32.2, but you'll have requests 2.22.0 which is incompatible.
ERROR: sentry-sdk 2.13.0 has requirement urllib3>=1.26.11, but you'll have urllib3 1.25.8 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset_name = "OpenAssistant/oasst1"
dataset = load_dataset(dataset_name, split="train")

/home/ubuntuadmin/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 4401/4401 [00:00<00:00, 212900.73 examples/s]


In [3]:
df = dataset.to_pandas()

def preprocess_data(df):
    df_assistant = df[(df.role == "assistant") & (df["rank"] == 0.0)].copy()
    df_assistant["output"] = df_assistant["text"].values

    df_prompter = df[(df.role == "prompter")].copy()
    df_prompter = df_prompter.set_index("message_id")

    inputs = []
    parent_ids = []

    for _, row in df_assistant.iterrows():
        input = df_prompter.loc[row.parent_id]

        inputs.append(input.text)
        parent_ids.append(input.parent_id)

    df_assistant["instruction"] = inputs
    df_assistant["parent_id"] = parent_ids

    df_assistant = df_assistant[df_assistant.lang == "en"]

    df_assistant = df_assistant[
        ["instruction", "output", "message_id", "parent_id"]
    ].rename(
        columns={"message_id": "id"}
    )

    return df_assistant

dataset_preprocessed = preprocess_data(df)

In [4]:
dataset_preprocessed.instruction.iloc[0]

'Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.'

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "microsoft/Phi-3.5-mini-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.52s/it]


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules
)

In [8]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 8
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    # gradient_checkpointing=True,
)

In [9]:
from datasets import Dataset
dataset_new = Dataset.from_pandas(dataset_preprocessed)
dataset = dataset_new.map(lambda data: {'text': [f'[INST] ' + prompt + ' [/INST] ' + response for prompt, response in zip(data['instruction'], data['output'])]}, batched=True)

Map:   0%|          | 0/7856 [00:00<?, ? examples/s]

Map: 100%|██████████| 7856/7856 [00:00<00:00, 99629.16 examples/s]


In [10]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/ubuntuadmin/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntuadmin/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntuadmin/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 7856/7856 [00:01<00:00, 5206.55 examples/s]
/home/ubuntuadmin/.local/lib/python3.8/site-packages/trl/trainer/sft_tra

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc

You are not running the flash-attention implementation, expect numerical differences.


Step,Training Loss
10,1.316600
20,1.185200
30,1.261600
40,1.560700
50,1.699900
60,1.202600
70,1.159200
80,1.211700
90,1.583300
100,1.682900


TrainOutput(global_step=500, training_loss=1.365170877456665, metrics={'train_runtime': 3098.742, 'train_samples_per_second': 2.582, 'train_steps_per_second': 0.161, 'total_flos': 3.4409411640274944e+16, 'train_loss': 1.365170877456665, 'epoch': 1.0183299389002036})

In [12]:
from transformers import pipeline

In [21]:
prompt = "Do you even know what L'Hopital's rule is?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'].split('.'))

["[INST] Do you even know what L'Hopital's rule is? [/INST] Yes, I do", " L'Hopital's rule is a mathematical theorem that states that if the limit of a function is of the form 0/0 or ∞/∞, then the limit of the function can be found by taking the derivative of the numerator and the derivative of the denominator and then taking the limit of the new function", ' This rule is useful in calculus and can be used to find limits of functions that would otherwise be difficult to evaluate', "\n\nFor example, if we have the function f(x) = (sin(x) - x) / x^3 and we want to find the limit as x approaches 0, we can use L'Hopital's rule to find that the limit is -1/6", ' This is because the limit of the numerator is 0 and the limit of the denominator']


In [16]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()
login(token=os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/ubuntuadmin/.cache/huggingface/token
Login successful


In [18]:
new_model = "phi3_custom_oasst1_v1"
model.save_pretrained(new_model)

In [22]:
tokenizer.save_pretrained(new_model)

('phi3_custom_oasst1_v1/tokenizer_config.json',
 'phi3_custom_oasst1_v1/special_tokens_map.json',
 'phi3_custom_oasst1_v1/tokenizer.json')

In [19]:
model.push_to_hub("phi3_custom_oasst1_v1")

model.safetensors: 100%|██████████| 2.97G/2.97G [01:40<00:00, 29.6MB/s]


CommitInfo(commit_url='https://huggingface.co/HimankJ/phi3_custom_oasst1_v1/commit/6cffbef42be9dcdc91042fce68468e6e5a3ad63a', commit_message='Upload Phi3ForCausalLM', commit_description='', oid='6cffbef42be9dcdc91042fce68468e6e5a3ad63a', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
tokenizer.push_to_hub("phi3_custom_oasst1_v1")

CommitInfo(commit_url='https://huggingface.co/HimankJ/phi3_custom_oasst1_v1/commit/2950e42ef316d7c56db578db09d64e28603a8151', commit_message='Upload tokenizer', commit_description='', oid='2950e42ef316d7c56db578db09d64e28603a8151', pr_url=None, pr_revision=None, pr_num=None)